In [ ]:
# Cell 1: Cài đặt
!pip install ollama fastapi uvicorn pydantic nest-asyncio pyngrok colab-xterm
!curl -fsSL https://ollama.com/install.sh | sh


In [ ]:
# Cell 2: Khởi chạy mọi thứ (Ollama + Tải model + FastAPI)

import os
import subprocess
import threading
import time
import ollama
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
import nest_asyncio

# --- 1. Khởi chạy "Bộ não" OLLAMA (Cổng 11434) ---
env = os.environ.copy()
env["OLLAMA_HOST"] = "0.0.0.0"
env["OLLAMA_ORIGINS"] = "*"

def run_ollama_serve():
    print("Starting Ollama server...")
    subprocess.Popen(["ollama", "serve"], env=env, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print("Ollama server process started.")

ollama_thread = threading.Thread(target=run_ollama_serve)
ollama_thread.start()
time.sleep(5) # Đợi Ollama khởi động

# --- 2. TẢI MÔ HÌNH  ---
try:
    print("Pulling model 'llama3'. This may take a few minutes...")
    # Lệnh này sẽ chạy và chờ cho đến khi tải xong
    subprocess.run(["ollama", "pull", "llama3"], check=True)
    print("Model 'llama3' pulled successfully.")
except Exception as e:
    print(f"Error pulling model: {e}")
    # Nếu lỗi ở đây, bạn cần kiểm tra lại

# --- 3. Định nghĩa "Giao diện API" FASTAPI (Cổng 8000) ---
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Khởi tạo client Ollama
ollama_client = ollama.Client(host='http://localhost:11434')

# Cấu trúc data mà Streamlit sẽ gửi
class ItineraryRequest(BaseModel):
    origin: str
    destination: str
    duration_days: int
    interests: str
    pace: str

@app.post("/generate_itinerary")
async def generate_itinerary(request: ItineraryRequest):
    print(f"Received request for {request.destination}")

    # Prompt (Lời nhắc) chi tiết
    prompt = f"""
    Bạn là một chuyên gia lập kế hoạch du lịch. Hãy tạo một lịch trình du lịch chi tiết cho chuyến đi {request.duration_days} ngày đến {request.destination}.
    Thông tin chuyến đi:
    - Khởi hành từ: {request.origin}
    - Sở thích: {request.interests}
    - Tốc độ chuyến đi: {request.pace}
    Yêu cầu đầu ra:
    - Chỉ trả về nội dung lịch trình.
    - Định dạng nghiêm ngặt bằng Markdown.
    - Phân chia theo từng ngày (ví dụ: ## Ngày 1: Tên chủ đề).
    - Mỗi ngày phải có ba phần: ### Buổi sáng, ### Buổi chiều, và ### Buổi tối.
    - Cung cấp giải thích ngắn gọn (1-2 câu) cho mỗi hoạt động.
    """

    try:
        response = ollama_client.chat(
            model='llama3', # Dùng mô hình đã tải
            messages=[{'role': 'user', 'content': prompt}]
        )
        itinerary_text = response['message']['content']
        return {"itinerary": itinerary_text}
    except Exception as e:
        print(f"Error calling Ollama: {e}")
        return {"error": "Failed to generate itinerary from Ollama."}

# --- 4. Chạy máy chủ FastAPI (Cổng 8000) ---
def run_fastapi_server():
    print("Starting FastAPI server on port 8000...")
    uvicorn.run(app, host="0.0.0.0", port=8000)

# nest_asyncio cho phép uvicorn chạy trong môi trường notebook
nest_asyncio.apply()

fastapi_thread = threading.Thread(target=run_fastapi_server)
fastapi_thread.start()
print("Backend setup complete. Both Ollama and FastAPI are running.")

Khi thanh terminal hiện ra nhấn
ssh -p 443 -R0:localhost:8000 qr@a.pinggy.io
để tiếp tục


In [ ]:
# Cell 3: Khởi động Terminal
%load_ext colabxterm
%xterm